# DLC/ANIPOSE PIPELINE

Automatically converts fly videos to estimated 3D coordinates using DeepLabCut and Anipose

In [1]:
# Next two lines are for dev purposes only 
%load_ext autoreload 
%autoreload 2

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("Logging works :)")
import pandas as pd
import numpy as np
from pipeline.pipeline_step_1 import run_preprocessing, analyze_new
from pathlib import Path
import glob
import os
from config import settings
COMMON_FILES = Path(settings.common_files)

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Loading DLC 2.2.3...


c:\Users\SthanuKumarD\Anaconda3\envs\DEEPLABCUT\lib\site-packages\deeplabcut\__init__.py:81: UserWarning: 
        As PyTorch is not installed, unsupervised identity learning will not be available.
        Please run `pip install torch`, or ignore this warning.
        
  warnings.warn(


### STEP 0: Configuration

- Set all necessary variables (videos path and root path) in `pipeline/config.yml`!

#### Override
If you would like to set these values in the noetbook, you can instead 
- Add filepath to DLC network config files (e.g `dlc_networks.yml`)
- Add filepath to folder with experiment videos

In [ ]:
"""NOTE: this is just for overriding the config parameters within notebook"""

# Set the filepath to the genotype folder containing videos
# videos = Path(r"\\mpfi.org\public\sb-lab\DLC_pipeline_Dummy\0_QualityCtrl\Raw_vid_test_new_Code_2\40_BDN1_CsChrim_decap")

# root directory of project (usually overall experiment folder or similar)
# root = Path(r'\\mpfi.org\public\sb-lab\DLC_pipeline_Dummy\0_QualityCtrl') # Root directory of experiment/run, typically genotype or some other overarching directory. 
# Videos must be in dir or a subdir of root 

### STEP 1: Running DeepLabCut

This will run on a directory with fly video files and generate DLC pose estimations, outputting to the same directory.

##### Processing done:
- DeepLabCut `analyze_videos`
- DeepLabCut `filterpredictions`

In [ ]:
analyze_new()
print("Finished DLC analysis...")

### Step 1b [only for SS files]

In [ ]:
### NOTE: run only once per folder; DONOT rerum
# TODO: check if files are preprocessed before appending extr columns
ref_cols = pd.read_csv(COMMON_FILES.joinpath("SS_preprocess_extracols.csv"), header = [0,1,2])
path = Path(settings.videos_path)
for fly_folder in os.listdir(path):
    fly_path = path / fly_folder
    if fly_folder.startswith('N'):
        for cam_file in os.listdir(Path(str(fly_path) + '/SS')):
            
            if cam_file.startswith('H') and cam_file.endswith('_filtered.csv'):
                print(f"Processing file: {cam_file} in {fly_path}")
                camH = pd.read_csv(Path(str(fly_path) + '/SS') / cam_file, header = [0,1,2])
                camH_new = pd.concat([camH, ref_cols], axis = 1)
                print(camH.shape, camH_new.shape)
                camH_new.to_csv(Path(str(fly_path) + '/SS') / camH, index = False) ## note: replaces the original file 

### STEP 2: Preprocessing for anipose

##### Processing done:
- Fix points
- Remove columns
- Rename (GenotypeFly#-CamName)
- Convert to .h5
- Generate anipose file structure

In [2]:
run_preprocessing()

INFO:pipeline.pipeline_step_1:Processing A-04112025165008-0000DLC_resnet101_SS_CamAMay22shuffle1_500000_filtered.csv
INFO:pipeline.pipeline_step_1: Running `Fix points` preprocessing...
INFO:pipeline.pipeline_step_1:Running `Remove cols` preprocessing...
INFO:pipeline.pipeline_step_1:Processing B-04112025165011-0000DLC_resnet101_SS_CamBMay22shuffle1_500000_filtered.csv
INFO:pipeline.pipeline_step_1: Running `Fix points` preprocessing...
INFO:pipeline.pipeline_step_1:Running `Remove cols` preprocessing...
INFO:pipeline.pipeline_step_1:camName `B`, removing cols starting with `L-`
INFO:pipeline.pipeline_step_1:Processing C-04112025165014-0000DLC_resnet101_SS_CamCMay22shuffle1_1000000_filtered.csv
INFO:pipeline.pipeline_step_1: Running `Fix points` preprocessing...
INFO:pipeline.pipeline_step_1:Running `Remove cols` preprocessing...
INFO:pipeline.pipeline_step_1:Processing D-04112025165009-0000DLC_resnet101_SS_CamDMay23shuffle1_500000_filtered.csv
INFO:pipeline.pipeline_step_1: Running `F

Cschrimson_decap_BDN2_SS_04112025_N1
Genotype for N1 is Cschrimson_decap_BDN2_SS_04112025_N1
Cschrimson_decap_BDN2_SS_04112025_N2
Genotype for N2 is Cschrimson_decap_BDN2_SS_04112025_N2
Cschrimson_decap_BDN2_SS_04112025_N3
Genotype for N3 is Cschrimson_decap_BDN2_SS_04112025_N3
Cschrimson_decap_BDN2_SS_04112025_N4
Genotype for N4 is Cschrimson_decap_BDN2_SS_04112025_N4
Cschrimson_decap_BDN2_SS_04112025_N5
Genotype for N5 is Cschrimson_decap_BDN2_SS_04112025_N5


INFO:pipeline.pipeline_step_1: Creating new directory Z:\NehaSapkal_new\Ball_system_data\bpn_project_pc1\BDN2_till04112025\Cschrimson_decap_BDN2_SS_04112025\anipose\SS\set1
INFO:pipeline.pipeline_step_1: Creating new directory Z:\NehaSapkal_new\Ball_system_data\bpn_project_pc1\BDN2_till04112025\Cschrimson_decap_BDN2_SS_04112025\anipose\SS\set1\calibration
INFO:pipeline.pipeline_step_1:Copying file \\mpfi.org\public\sb-lab\BallSystem_CalibrationMov\2025\02252025\set2-better\calibration\detections.pickle to Z:\NehaSapkal_new\Ball_system_data\bpn_project_pc1\BDN2_till04112025\Cschrimson_decap_BDN2_SS_04112025\anipose\SS\set1\calibration\detections.pickle
INFO:pipeline.pipeline_step_1:Copying file \\mpfi.org\public\sb-lab\BallSystem_CalibrationMov\2025\02252025\set2-better\calibration\calibration.toml to Z:\NehaSapkal_new\Ball_system_data\bpn_project_pc1\BDN2_till04112025\Cschrimson_decap_BDN2_SS_04112025\anipose\SS\set1\calibration\calibration.toml
INFO:pipeline.pipeline_step_1: Creating 

Finished preprocessing...
